In [1]:
import numpy as np
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA

---
Dimensionnality Reduction
---
---
Training millions of features is very slow weed could need to reduce the size of the features to speed up the training, in particular when it could be done without so much loss of  information

1.Curse of dimensionality
---
If the dataset has a lot of dimension, each instance will be unavoidable very sparse (each instance will be at great distance from each other). Consequently, the model will be less accurate than a model with lower dimension and the risk of overfitting is increasing.

2.Main approaches for dimensionality reduction
----

### a) Projection
When a training of dimension n set has an almost constant feature in the nth dimension for example, it's possible to project all the features in a dimension n-1. Example in 3D XYZ, if all values are almost constant regarding Z, the dataset could be projected on XY plan

### b) Manifold Learning
It's the idea that higher dimension lie close to a manifold of lower dimension and that they could be equalized

3.Dimensionality Reduction Algorithm
---

### a) Principal Component Analysis (PCA)
The main idea is to preserve as much as possible the variance when in dimension n when projecting on dimension n-1. Then it finds a second axis orthogonal to the fisrt axis with the second maximal variance etc..

Principal components could be find using Singular Value Decomposition (SVD). But be careful PCA assume that the dataset is centered

#### PCA Using numpy

#### For Calculating the Singular Value Decomposition

In [ ]:
X_centered = X - X.mean(axis=0) # Dataset is centered
U, s, Vt = p.linalg.svd(X_centered)
c1 = Vt.T[:, 0]
c2 = Vt.T[:, 1]

#### Projection down to d dimensions  
To project X on the hyperplane of d dimension, X **centered** is multiplied by $W_{d}$ the matrix of the d first columns of V.

In [ ]:
d = 2 # Example with d=2
Wd = Vt.T[:, :d]
Xd_proj = X_centered.dot(Wd)

#### PCA Using sklearn
No need to center X first

In [ ]:
pca = PCA(n_components = 2) # 2 equal the dimension of the proj
X2D = pca.fit_transform(X)

#### Explained Variance Ratio
explained_variance_ratio_ explicits the ratio of the variance that lies along the d first components of the PCA. (if the sum is closed to 100 the dimensionality waws the proper thing to do)

In [ ]:
pca.explained_variance_ratio_

To keep 95% of the variance with dimensionality reduction you have 2 solutions :

In [ ]:
# n°1 solution
pca = PCA()
pca.fit(X_train)
cumsum = np.cumsum(pca.explained_variance_ration_)
d = np.argmax(cumsum=0.95)+1
pca.n_components = d
pca.fit_transform(X)

In [ ]:
# n°2 solution
pca = PCA(n_components=0.95) # n_components with a ratio between 0 and 1 indicating 
pca.fit_transform(X)         #the variance you want to preserve

#### PCA Compression
You can reconstruct the original dataset with a bit of lost information(among the percent of variance lost)  
$X_{recovered} = X_{d-proj}W_{d}^{T}$

In [ ]:
pca = PCA(n_components = 154)
X_reduced = pca.fit_transform(X)
X_recovered = pca.inverse_transform(X_reduced)

#### Randomized PCA
You can quickly find approximate PCA with stochastic approach, by setting the svd_solver to randomized,  
 - by default svd_solver=auto -> uses randomized if $dimension>500$ and d<0.8\*dimension.  
 - set svd_solver to full if you want the compute full SVD

In [ ]:
pca = PCA(n_components = 154, svd_solver="randomized")
X_reduced = pca.fit_transform(X)

#### Incremental PCA
On large dataset it could be complicated to compute PCA on all the dataset. Fortunately, Incremental PCA is used splitting the dataset in mini batches for computing PCA. It is useful for :
- Large dataset
- Online dataset (with on the fly data)

In [ ]:
n_batches = 100
inc_pca = IncrementalPCA(n_components=154)

# PCA is fitted for each mini batches
for X_batch in np.array.split(X_train, n_batches):
    inc_pca.fit(X_batch)

# Then X is reduced with X train
X_reduced = inc_pca.transform(X_train)

If you manipulate large array stored on disk file it's still possible to use IncrementalPCA

In [ ]:
X_mm = np.memmap(filename, dtype="float32", mode="readonly", shape=(m,n))

batch_size = m//n_batches
inc_pca = IncrementalPCA(n_components=154, batch_size=batch_size)
inc_pca.fit(X_mm)

#### Kernel PCA
Kernel PCA is an unsupervised learning technique

In [ ]:
rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.04)
X_reduced = rbf_pca.fit_transform(X)

#### Finding best Kernel hyperparameters n°1
For Finding the best KPCA we could proceed as follow to automate the task

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ("kpca", KernelPCA(n_components=2)),
    ("log_reg", LogisticRegression())
])

param_grid = [{
    "kpca_gamma" : np.linspace(0.03, 0.05, 10),
    "kpca_kernel" : ["rbf", "sigmoid"]
}]

grid_search = GridSearchCV(clf, param_grid, cv=3)
grid_search.fit(X,y)

print(grid_search.best_params_)

#### Finding best Kernel hyperparameters n°2
To find the best kPCA we could also use an unsupervised technique and keeping the kPCA with the lowest reconstruction error. However the reconstruction is not the same as linear PCA with inverse transform

In [ ]:
from sklearn.metrics import mean_squared_error
# By setting fit_inverse_transform we specify we want to reconstruct the reduction
rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.0433, fit_inverse_transform=True)

X_reduced = rbf_pca.fit_transform(X)
X_preimage = rbf_pca.inverse_transform(X_reduced)

# The goal is to minimize mean_squared_error
mean_squared_error(X, X_preimage)

### b) Locally Linear Embedding (LLE)
Nonlinear Dimensionnality reduction: looking how neighboors of each instance are linearly related and then tried to reduce the dimension while preserving the linear relations.

In [ ]:
from sklearn.manifold import LocallyLinearEmbedding

lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10)
X_reduced = lle.fit_transform(X)

### c) Random projections
Random linear projections preserves well the distance

### d) Multidimensional Scaling
Reduces the dimensionality while trying to preserve the distances between instances

### e) Isomap
Creates a graph by connecting each instance to its nearest neighbors, thn reduce dimensionality while trying preserving geodesic distances between them

### f) t-Distributed Stochastic Neighbor Embedding (t-SNE)
Reduces dimensionality while triyng to keep similar instances close and dissimilar instances apart.

### g) Linear Discriminant Analysis (LDA) 
Classification Algo, during training learns the most discriminative axes between the classes, and these axezs can be used to form hyperplane